<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/parsing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path.append('/content/')
import Drain
import pandas as pd


In [ ]:
input_dir  = '/content/'  # The input directory of log file
output_dir = 'Drain_result/'  # The output directory of parsing results
log_file   = 'HDFS_2k.log'  # The input log file name
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
# Regular expression list for optional preprocessing (default: [])
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Similarity threshold
depth      = 4  # Depth of all leaf nodes

parser = Drain.LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex)
parser.parse(log_file)

Parsing file: /content/HDFS_2k.log
Total size after encoding is 2000 2000
Parsing done. [Time taken: 0:00:00.352760]


In [ ]:
df_parsedlog = pd.read_csv('/content/Drain_result/HDFS_2k.log_structured.csv')

In [ ]:
df_parsedlog.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203615,148,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_38865049064139...,dc2c74b7,PacketResponder <*> for block <*> terminating,[]
1,2,81109,203807,222,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-6952295868487...,dc2c74b7,PacketResponder <*> for block <*> terminating,[]
2,3,81109,204005,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,[]
3,4,81109,204015,308,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_82291938032499...,dc2c74b7,PacketResponder <*> for block <*> terminating,[]
4,5,81109,204106,329,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-6670958622368...,dc2c74b7,PacketResponder <*> for block <*> terminating,[]


In [ ]:
# Get the unique items of the 'EventId' column
unique_EventId = df_parsedlog['EventId'].unique()

print(unique_EventId)

['dc2c74b7' '5d5de21c' 'e3df2680' '09a53393' '3d91fa85' '32777b38'
 'dba996ef' '626085d5' '81cee340' 'd63ef163' '40651754' '04137b95'
 'd6b7b743' '98012f03' '728076ac' '415a1760']


In [ ]:
# Define the mapping dictionary automatically
mapping = {item: index for index, item in enumerate(unique_EventId)}

print(mapping)

{'dc2c74b7': 0, '5d5de21c': 1, 'e3df2680': 2, '09a53393': 3, '3d91fa85': 4, '32777b38': 5, 'dba996ef': 6, '626085d5': 7, '81cee340': 8, 'd63ef163': 9, '40651754': 10, '04137b95': 11, 'd6b7b743': 12, '98012f03': 13, '728076ac': 14, '415a1760': 15}


In [ ]:
# Replace the values using the mapping dictionary
df_parsedlog['EventId'] = df_parsedlog['EventId'].replace(mapping)

print(df_parsedlog)

      LineId   Date    Time    Pid Level                     Component  \
0          1  81109  203615    148  INFO  dfs.DataNode$PacketResponder   
1          2  81109  203807    222  INFO  dfs.DataNode$PacketResponder   
2          3  81109  204005     35  INFO              dfs.FSNamesystem   
3          4  81109  204015    308  INFO  dfs.DataNode$PacketResponder   
4          5  81109  204106    329  INFO  dfs.DataNode$PacketResponder   
...      ...    ...     ...    ...   ...                           ...   
1995    1996  81111  101621  24902  INFO      dfs.DataNode$DataXceiver   
1996    1997  81111  101735  26595  INFO  dfs.DataNode$PacketResponder   
1997    1998  81111  101804  26494  INFO      dfs.DataNode$DataXceiver   
1998    1999  81111  101954  26414  INFO  dfs.DataNode$PacketResponder   
1999    2000  81111  102017  26347  INFO      dfs.DataNode$DataXceiver   

                                                Content  EventId  \
0     PacketResponder 1 for block blk_38865

In [ ]:
df_parsedlog['EventId']

0       0
1       0
2       1
3       0
4       0
       ..
1995    3
1996    2
1997    3
1998    0
1999    3
Name: EventId, Length: 2000, dtype: int64

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from collections import OrderedDict

def hdfs_sampling(log_file, window='session', window_size=0):
    assert window == 'session', "Only window=session is supported for HDFS dataset."
    print("Loading", log_file)
    struct_log = pd.read_csv(log_file, engine='c',
            na_filter=False, memory_map=True)
    data_dict = OrderedDict()
    for idx, row in struct_log.iterrows():
        blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
        blkId_set = set(blkId_list)
        for blk_Id in blkId_set:
            if not blk_Id in data_dict:
                data_dict[blk_Id] = []
            data_dict[blk_Id].append(row['EventId'])
    data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])
    data_df.to_csv("hdfs/HDFS_sequence.csv",index=None)

hdfs_sampling('hdfs/HDFS_100k.log_structured.csv')

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from collections import OrderedDict

In [ ]:
df_parsedlog['Content']

0       PacketResponder 1 for block blk_38865049064139...
1       PacketResponder 0 for block blk_-6952295868487...
2       BLOCK* NameSystem.addStoredBlock: blockMap upd...
3       PacketResponder 2 for block blk_82291938032499...
4       PacketResponder 2 for block blk_-6670958622368...
                              ...                        
1995    Receiving block blk_4198733391373026104 src: /...
1996    Received block blk_-5815145248455404269 of siz...
1997    Receiving block blk_-295306975763175640 src: /...
1998    PacketResponder 0 for block blk_52257196770490...
1999    Receiving block blk_4343207286455274569 src: /...
Name: Content, Length: 2000, dtype: object

In [ ]:
re.findall(r'(blk_-?\d+)', df_parsedlog['Content'][0])

['blk_38865049064139660']

In [ ]:
data_dict = OrderedDict()
for idx, row in df_parsedlog.iterrows():
    blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
    blkId_set = set(blkId_list)
    for blk_Id in blkId_set:
        if not blk_Id in data_dict:
            data_dict[blk_Id] = []
        data_dict[blk_Id].append(row['EventId'])

In [ ]:
data_dict

OrderedDict([('blk_38865049064139660', [0]),
             ('blk_-6952295868487656571', [0]),
             ('blk_7128370237687728475', [1]),
             ('blk_8229193803249955061', [0]),
             ('blk_-6670958622368987959', [0]),
             ('blk_3050920587428079149', [1]),
             ('blk_7888946331804732825', [1]),
             ('blk_2377150260128098806', [1]),
             ('blk_572492839287299681', [0]),
             ('blk_3587508140051953248', [2]),
             ('blk_5402003568334525940', [2]),
             ('blk_5792489080791696128', [3]),
             ('blk_1724757848743533110', [3]),
             ('blk_8015913224713045110', [1]),
             ('blk_-5623176793330377570', [3]),
             ('blk_-1727475099218615100', [4]),
             ('blk_5017373558217225674', [0]),
             ('blk_9212264480425680329', [2]),
             ('blk_-7878121102358435702', [4]),
             ('blk_4568434182693165548', [1]),
             ('blk_-5704899712662113150', [2]),
          

In [ ]:
data_dict.items()

odict_items([('blk_38865049064139660', [0]), ('blk_-6952295868487656571', [0]), ('blk_7128370237687728475', [1]), ('blk_8229193803249955061', [0]), ('blk_-6670958622368987959', [0]), ('blk_3050920587428079149', [1]), ('blk_7888946331804732825', [1]), ('blk_2377150260128098806', [1]), ('blk_572492839287299681', [0]), ('blk_3587508140051953248', [2]), ('blk_5402003568334525940', [2]), ('blk_5792489080791696128', [3]), ('blk_1724757848743533110', [3]), ('blk_8015913224713045110', [1]), ('blk_-5623176793330377570', [3]), ('blk_-1727475099218615100', [4]), ('blk_5017373558217225674', [0]), ('blk_9212264480425680329', [2]), ('blk_-7878121102358435702', [4]), ('blk_4568434182693165548', [1]), ('blk_-5704899712662113150', [2]), ('blk_-4794867979917102672', [1]), ('blk_8763662564934652249', [1]), ('blk_-5861636720645142679', [2]), ('blk_7453815855294711849', [1]), ('blk_-28342503914935090', [3]), ('blk_8291449241650212794', [2]), ('blk_-5319073033164653435', [4]), ('blk_-4980916519894289629', [

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from collections import OrderedDict

def hdfs_sampling(log_file, window='session', window_size=0):
    assert window == 'session', "Only window=session is supported for HDFS dataset."
    print("Loading", log_file)
    struct_log = pd.read_csv(log_file, engine='c',
            na_filter=False, memory_map=True)
    data_dict = OrderedDict()
    for idx, row in struct_log.iterrows():
        blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
        blkId_set = set(blkId_list)
        for blk_Id in blkId_set:
            if not blk_Id in data_dict:
                data_dict[blk_Id] = []
            data_dict[blk_Id].append(row['EventId'])
    data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])
    data_df.to_csv("hdfs/HDFS_sequence.csv",index=None)

hdfs_sampling('hdfs/HDFS_100k.log_structured.csv')